## Importing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

In [3]:
from utils.utils import *
import optuna
from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend
from experiments.experiments_gkan import ExperimentsGKAN
from experiments.experiments_mpnn import ExperimentsMPNN
import sympytorch

import warnings
warnings.filterwarnings("ignore")


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [4]:
import random

def set_pytorch_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    torch.use_deterministic_algorithms(True)
set_pytorch_seed(0)

## Utils

In [5]:
from models.utils.MPNN import MPNN
from models.baseline.MPNN_ODE import MPNN_ODE
from train_and_eval import eval_model
from datasets.SyntheticData import SyntheticData
from sympy import symbols, sin, summation, simplify
import networkx as nx
from torch_geometric.utils import from_networkx
from utils.utils import integrate
from torch_geometric.data import Data
from experiments.experiments_mpnn import activations
from models.utils.MLP import MLP

import json
import torch
from models.baseline.LLC import LLC_ODE
from models.utils.MPNN import MPNN
from models.baseline.LLC_Conv import Q_inter, Q_self
from experiments.experiments_mpnn import activations

In [6]:
from sympy import latex
from torch.utils.data import DataLoader

def get_model(g, h, message_passing=True, include_time=False, atol=1e-5, rtol=1e-5, integration_method = 'scipy_solver'):
    conv = MPNN(
        g_net = g,
        h_net = h, 
        message_passing=message_passing,
        include_time=include_time
    )
    
    symb = MPNN_ODE(
        conv=conv,
        model_path="./saved_models_optuna/tmp_symb",
        adjoint=True,
        integration_method=integration_method,
        atol=atol,
        rtol=rtol
    )
    
    symb = symb.eval()
    return symb


def make_callable(expr):
    free_syms = expr.free_symbols
    if not free_syms:
        # Expression is constant
        const_value = float(expr)
        return lambda x: torch.full((x.shape[0], 1), const_value, dtype=x.dtype, device=x.device)

    sym_module = sympytorch.SymPyModule(expressions=[expr])
    syms = {str(s) for s in free_syms}
    if {'x_i', 'x_j'} <= syms:
        return lambda x: sym_module(x_i=x[:, 0], x_j=x[:, 1])
    elif 'x_i' in syms:
        return lambda x: sym_module(x_i=x[:, 0])
    elif 'x_j' in syms:
        return lambda x: sym_module(x_j=x[:, 1])
    else:
        raise ValueError(f"Unexpected symbols in expression: {free_syms}")


def get_symb_test_error(g_symb, h_symb, test_set, message_passing=False, include_time=False, atol=1e-5, rtol=1e-5, scaler = None, inverse_scale=False, method='scipy_solver',
                        is_symb=True):
    
    if is_symb:
        if isinstance(g_symb, int):
            g_symb = sp.sympify(g_symb)
            
        if isinstance(h_symb, int):
            h_symb = sp.sympify(h_symb)

        g_symb = make_callable(g_symb)
        h_symb = make_callable(h_symb)
        
    test_losses = []
    
    for ts in test_set:
        symb = get_model(
            g=g_symb,
            h=h_symb,
            message_passing=message_passing,
            include_time=include_time,
            atol=atol,
            rtol=rtol,
            integration_method=method
        )
        
        collate_fn = lambda samples_list: samples_list
        test_loader = DataLoader(ts, batch_size=len(ts), shuffle=True, collate_fn=collate_fn)
        
        test_loss = eval_model(
            model=symb,
            valid_loader=test_loader,
            criterion=torch.nn.L1Loss(),
            scaler=scaler,
            inverse_scale=inverse_scale,
            pred_deriv=False
        )
        
        test_losses.append(test_loss)
    
    return test_losses



def get_test_set(dynamics, device='cuda', input_range=(0, 1), t_span = (0, 1), **integration_kwargs):
    seeds = [12345, 67890, 111213]
    
    graphs = [
        nx.barabasi_albert_graph(70, 3, seed=seeds[0]),      
        nx.watts_strogatz_graph(50, 6, 0.3, seed=seeds[1]),  
        nx.erdos_renyi_graph(100, 0.05, seed=seeds[2])        
    ]
    
    test_set = []
    for i, graph in enumerate(graphs):
        snapshots = integrate_test_set(
            graph=graph,
            dynamics=dynamics,
            seed=seeds[i],
            device=device,
            input_range=input_range,
            t_span=t_span,
            **integration_kwargs
        )
        test_set.append(snapshots)
    
    return test_set
    


def integrate_test_set(graph, dynamics, seed=12345, device='cuda', input_range = (0, 1), t_span = (0, 1), **integration_kwargs):
    # graph = nx.barabasi_albert_graph(100, 3, seed=seed)
    edge_index = from_networkx(graph).edge_index
    edge_index = edge_index.to(torch.device(device))
    rng = np.random.default_rng(seed=seed)
    
    data, t = integrate(
        input_range=input_range,
        t_span = t_span,
        t_eval_steps=300,
        dynamics=dynamics,
        device=device,
        graph=graph,
        rng = rng,
        **integration_kwargs
    )
    
    snapshot = Data(
        x = data[0].unsqueeze(0),
        y = data[1:],
        edge_index=edge_index,
        edge_attr=None,
        t_span = t
    )
    
    return [snapshot]


def build_model_from_file(model_path, message_passing=False, include_time=False, method='dopri5', adjoint=True, atol=1e-5, rtol=1e-5):
    best_params_file = f"{model_path}/best_params.json"
    best_state_path = f"{model_path}/mpnn/state_dict.pth"
    with open(best_params_file, 'r') as f:
        best_hyperparams = json.load(f)
    
    in_dim = 1
    
    hidden_layers = [best_hyperparams["hidden_dims_g_net"] for _ in range(best_hyperparams["n_hidden_layers_g_net"])]
    hidden_layers = [2*in_dim] + hidden_layers + [in_dim]    
    # g_net
    g_net = MLP(
        hidden_layers=hidden_layers,
        af = activations[best_hyperparams['af_g_net']],
        dropout_rate=best_hyperparams['drop_p_g_net'],
    )
    
    time_dim = 1 if include_time else 0
    in_dim_h = 2 if message_passing else 1
    in_dim_h += time_dim
    hidden_layers = [best_hyperparams["hidden_dims_h_net"] for _ in range(best_hyperparams["n_hidden_layers_h_net"])]
    hidden_layers = [in_dim_h] + hidden_layers + [in_dim] 
    
    
    # h_net
    h_net = MLP(
        hidden_layers=hidden_layers,
        af = activations[best_hyperparams['af_h_net']],
        dropout_rate=best_hyperparams['drop_p_h_net'],
    )
    
    mpnn = MPNN(
        h_net=h_net,
        g_net=g_net,
        message_passing=message_passing,
        include_time=include_time
    )
    
    model = MPNN_ODE(
        conv=mpnn,
        model_path='./saved_models_optuna/tmp',
        integration_method=method,
        adjoint=adjoint,
        atol=atol,
        rtol=rtol
    )
    
    model = model.to(torch.device('cuda'))
    model.load_state_dict(torch.load(best_state_path, weights_only=False, map_location=torch.device('cuda')))
    
    return model


def build_model_from_file_llc(model_path, message_passing=False, include_time=False, method='dopri5', adjoint=True, atol=1e-5, rtol=1e-5):

    best_params_file = f"{model_path}/best_params.json"
    best_state_path = f"{model_path}/llc/state_dict.pth"

    with open(best_params_file, 'r') as f:
        best_hyperparams = json.load(f)

    in_dim = 1
    time_dim = 1 if include_time else 0

    # === g_net config (for Q_inter) ===
    def build_q_layer_config(prefix):
        n_layers = best_hyperparams[f'n_hidden_layers_{prefix}']
        hidden_dim = best_hyperparams[f'hidden_dims_{prefix}']
        activation = activations[best_hyperparams[f'af_{prefix}']]
        dropout = best_hyperparams[f'drop_p_{prefix}']

        if prefix == "g0":
            input_dim = 2 * in_dim
        else:
            input_dim = in_dim 

        hidden_layes = [hidden_dim] * n_layers
        layers = [input_dim] + hidden_layes + [in_dim]
        return {
            f'hidden_layers_{prefix}': layers,
            f'af_{prefix}': activation,
            f'dr_{prefix}': dropout
        }

    g0_config = build_q_layer_config("g0")
    g1_config = build_q_layer_config("g1")
    g2_config = build_q_layer_config("g2")
    g_net = Q_inter(**{**g0_config, **g1_config, **g2_config})

    # === h_net config (for Q_self) ===
    in_dim_h = 2 * in_dim if message_passing else in_dim
    in_dim_h += time_dim
    hidden_dim_h = best_hyperparams["hidden_dims_h_net"]
    n_layers_h = best_hyperparams["n_hidden_layers_h_net"]
    hidden_layers_h = [hidden_dim_h] * n_layers_h
    layers_h = [in_dim_h] + hidden_layers_h + [in_dim]

    h_net = Q_self(
        hidden_layers=layers_h,
        af=activations[best_hyperparams['af_h_net']],
        dropout_rate=best_hyperparams['drop_p_h_net']
    )

    # === Full MPNN and ODE wrapper ===
    mpnn = MPNN(
        h_net=h_net,
        g_net=g_net,
        message_passing=message_passing,
        include_time=include_time
    )

    model = LLC_ODE(
        conv=mpnn,
        model_path='./saved_models_optuna/tmp',
        adjoint=adjoint,
        integration_method=method,
        atol=atol,
        rtol=rtol
    )

    model = model.to(torch.device('cuda'))
    model.load_state_dict(torch.load(best_state_path, weights_only=False, map_location=torch.device('cuda')))

    return model


def valid_symb_model(
    config,
    model_path_gkan,
    device='cuda',
    atol=1e-5,
    rtol=1e-5,
    method='dopri5',
    sample_size=10000
):
    seed = 9999
    graph = nx.barabasi_albert_graph(100, 3, seed=seed)

    # Prepare validation/test set
    valid_set = integrate_test_set(
        graph=graph,
        dynamics=config['name'],
        seed=seed,
        device=device,
        input_range=config['input_range'],
        t_span=(0, 1),
        **config['integration_kwargs']
    )

    # Helper to compute validation loss
    def evaluate_model(g_symb, h_symb, is_symb=True):
        errs = get_symb_test_error(
            g_symb=g_symb,
            h_symb=h_symb,
            test_set=[valid_set],
            message_passing=False,
            include_time=False,
            method=method,
            atol=atol,
            rtol=rtol,
            is_symb=is_symb
        )
        return errs[0]

    # Helper to fit model for current config
    def fit_single_model(param1, param2):
        print(f"Fitting black-box model with {param1} and {param2} iterations")
        pysr_model = lambda: get_pysr_model(
            model_selection=param1, 
            n_iterations=param2,
            # parallelism="serial",
            # random_state = seed,
            # deterministic = True
        )
        _, g_symb, h_symb, _ = fit_mpnn(
            device=device,
            model_path=model_path_gkan,
            pysr_model=pysr_model,
            sample_size=sample_size,
            message_passing=False,
            verbose=False
        )
        
        return g_symb, h_symb

    param_grid = (["score", "accuracy"], [50, 100, 200])
    search_space = [(mod, val) for mod in param_grid[0] for val in param_grid[1]]
    valid_losses = []
    
    for mod, val in search_space:
        g_symb, h_symb = fit_single_model(mod, val)
        try:
            loss = evaluate_model(g_symb, h_symb, is_symb=True)
        except AssertionError:
            loss = 1e8
        valid_losses.append({'model_selection': mod, 'param': val, 'valid_loss': loss})
    
    best = min(valid_losses, key=lambda x: x['valid_loss'])    
    
    print(f"Refitting best model with {best}")
    
    gkan_symb, symb_g, symb_h, exec_time = fit_mpnn(
        model_path=model_path_gkan,
        device=device,
        pysr_model=lambda: get_pysr_model(
            model_selection=best['model_selection'],
            n_iterations=best['param'],
            # parallelism="serial",
            # random_state = seed,
            # deterministic = True
        ),
        sample_size=sample_size,
        message_passing=False,
        verbose=True,
        include_time=False
    )
    
    return gkan_symb, symb_g, symb_h, exec_time


def post_process_mpnn(
    config,
    model_path, 
    test_set, 
    device='cuda',
    sample_size=10000,
    message_passing=False, 
    include_time=False,
    atol=1e-5,
    rtol=1e-5,
    method='dopri5',
    scaler=None,
    inverse_scale=False,
    adjoint=True,
    eval_model=True,
    model_type="MPNN"
):
    
    results_dict = {}
    
    def print_symb_error(g_symb, h_symb, txt="symbolic formula", is_symb=True):
        try:
            test_losses_symb = get_symb_test_error(
                g_symb=g_symb,
                h_symb=h_symb,
                test_set=test_set,
                message_passing=message_passing,
                include_time=include_time,
                atol=atol,
                rtol=rtol,
                method=method,
                scaler=scaler,
                inverse_scale=inverse_scale,
                is_symb=is_symb
            )

            ts_mean = np.mean(test_losses_symb)
            ts_var = np.var(test_losses_symb)
            ts_std = np.std(test_losses_symb)
            
            print(f"Mean Test loss of {txt}: {ts_mean}")
            print(f"Var Test loss of {txt}: {ts_var}")
            print(f"Std Test loss of {txt}: {ts_std}")
            
            return ts_mean, ts_var, ts_std
        except AssertionError:
            print("Evaluation failed!")
            return np.inf, np.inf, np.inf
        
    
    print("Black-Box fitting \n")
    bb_symb, bb_g_symb, bb_h_symb, exec_time = valid_symb_model(
        config=config,
        model_path_gkan=f"{model_path}/mpnn" if model_type == 'MPNN' else f"{model_path}/llc",
        device=device,
        atol=atol,
        rtol=rtol,
        method=method,
        sample_size = sample_size
    )
    
    print(latex(quantise(bb_symb)))
    ts_mean_bb, ts_var_bb, ts_std_bb = print_symb_error(g_symb=bb_g_symb, h_symb=bb_h_symb)
    
    results_dict["black_box_symb_quant"] = str(quantise(bb_symb))
    results_dict["black_box_symb"] = str(bb_symb)
    results_dict["black_box_symb_test_MAE"] = ts_mean_bb
    results_dict["black_box_symb_test_Var"] = ts_var_bb
    results_dict["black_box_symb_test_Std"] = ts_std_bb
    results_dict["black_box_exec_time"] = exec_time
    
    if eval_model:
        print("Evaluate raw model\n")
        # Loading best model
        if model_type == "MPNN":
            best_model = build_model_from_file(
                model_path=model_path,
                message_passing=message_passing,
                include_time=include_time,
                method=method,
                adjoint=adjoint,
                atol=atol,
                rtol=rtol
            )
        elif model_type == "LLC":
            best_model = build_model_from_file_llc(
                model_path=model_path,
                message_passing=message_passing,
                include_time=include_time,
                method=method,
                adjoint=adjoint,
                rtol=rtol,
                atol=atol
            )
        else:
            raise NotImplementedError("Not supported model!")

        tot_params = sum(p.numel() for p in best_model.parameters() if p.requires_grad)
        print(f"Number of model's parameters: {tot_params}\n")
        results_dict["Number of params"] = tot_params

        best_model = best_model.eval()
        ts_mean_model, ts_var_model, ts_std_model = print_symb_error(
            g_symb=best_model.conv.model.g_net,
            h_symb=best_model.conv.model.h_net,
            txt="best model",
            is_symb=False
        )
        
        results_dict["model_test_MAE"] = ts_mean_model
        results_dict["model_test_Var"] = ts_var_model
        results_dict["model_test_Std"] = ts_std_model
    
    with open(f"{model_path}/post_process_res.json", 'w') as file:
        json.dump(results_dict, file, indent=4)   


def plot_predictions(y_true, y_pred, node_index = 0):
    plt.figure(figsize=(16, 8))
    plt.plot(y_true[:, node_index, :], label='y_true', marker='o')
    plt.plot(y_pred[:, node_index, :], label='y_pred', marker='o')
    plt.xlabel('Time step')
    plt.ylabel('Value')
    plt.title(f'y_true vs y_pred for Node {node_index}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


## LB Losses

### Kuramoto

In [7]:
kur_config = load_config("./configs/config_pred_deriv/config_ic1/config_kuramoto.yml")

KUR = get_test_set(
    dynamics=kur_config['name'],
    device='cuda',
    input_range=kur_config['input_range'],
    **kur_config['integration_kwargs']    
)

g_symb = lambda x: torch.sin(x[:, 1] - x[:, 0]).unsqueeze(-1)
h_symb = lambda x: 2.0 + 0.5 * x[:, 1].unsqueeze(-1)

test_losses = get_symb_test_error(
    g_symb=g_symb,
    h_symb=h_symb,
    test_set=KUR,
    message_passing=True,
    include_time=False,
    is_symb=False
)

ts_mean = np.mean(test_losses)
ts_var = np.var(test_losses)
ts_std = np.std(test_losses)

print(f"Mean Test loss of symbolic formula: {ts_mean}")
print(f"Var Test loss of symbolic formula: {ts_var}")
print(f"Std Test loss of symbolic formula: {ts_std}")



Mean Test loss of symbolic formula: 1.352664670169664e-05
Var Test loss of symbolic formula: 1.3517513389612395e-13
Std Test loss of symbolic formula: 3.676617112185112e-07


### Epidemics

In [8]:
epid_config = load_config("./configs/config_pred_deriv/config_ic1/config_epidemics.yml")

EPID = get_test_set(
    dynamics=epid_config['name'],
    device='cuda',
    input_range=epid_config['input_range'],
    **epid_config['integration_kwargs']    
)

g_symb = lambda x: 0.5*x[:, 1].unsqueeze(-1) * (1 - x[:, 0].unsqueeze(-1))
h_symb = lambda x: x[:, 1].unsqueeze(1) - 0.5 * x[:, 0].unsqueeze(-1)

test_losses = get_symb_test_error(
    g_symb=g_symb,
    h_symb=h_symb,
    test_set=EPID,
    message_passing=True,
    include_time=False,
    is_symb=False
)


ts_mean = np.mean(test_losses)
ts_var = np.var(test_losses)
ts_std = np.std(test_losses)

print(f"Mean Test loss of symbolic formula: {ts_mean}")
print(f"Var Test loss of symbolic formula: {ts_var}")
print(f"Std Test loss of symbolic formula: {ts_std}")


Mean Test loss of symbolic formula: 1.0729370387707604e-06
Var Test loss of symbolic formula: 8.017524161241833e-14
Std Test loss of symbolic formula: 2.831523293430911e-07


### Biochemical

In [9]:
bio_config = load_config("./configs/config_pred_deriv/config_ic1/config_biochemical.yml")

BIO = get_test_set(
    dynamics=bio_config['name'],
    device='cuda',
    input_range=bio_config['input_range'],
    **bio_config['integration_kwargs']    
)

g_symb = lambda x: (-0.5*x[:, 1] * x[:, 0]).unsqueeze(-1)
h_symb = lambda x: (1.0 - 0.5 * x[:, 0]).unsqueeze(-1)  + x[:, 1].unsqueeze(-1) 

test_losses = get_symb_test_error(
    g_symb=g_symb,
    h_symb=h_symb,
    test_set=BIO,
    message_passing=True,
    include_time=False,
    is_symb=False
)

ts_mean = np.mean(test_losses)
ts_var = np.var(test_losses)
ts_std = np.std(test_losses)

print(f"Mean Test loss of symbolic formula: {ts_mean}")
print(f"Var Test loss of symbolic formula: {ts_var}")
print(f"Std Test loss of symbolic formula: {ts_std}")


Mean Test loss of symbolic formula: 1.2020226639227378e-06
Var Test loss of symbolic formula: 7.199001650861301e-14
Std Test loss of symbolic formula: 2.683095535172257e-07


### Population

In [10]:
pop_config = load_config("./configs/config_pred_deriv/config_ic1/config_population.yml")

POP = get_test_set(
    dynamics=pop_config['name'],
    device='cuda',
    input_range=pop_config['input_range'],
    **pop_config['integration_kwargs']    
)

g_symb = lambda x: 0.2*torch.pow(x[:, 1].unsqueeze(-1), 3)
h_symb = lambda x: -0.5 * x[:, 0].unsqueeze(-1) + x[:, 1].unsqueeze(1) 

test_losses = get_symb_test_error(
    g_symb=g_symb,
    h_symb=h_symb,
    test_set=POP,
    message_passing=True,
    include_time=False,
    is_symb=False
)

ts_mean = np.mean(test_losses)
ts_var = np.var(test_losses)
ts_std = np.std(test_losses)

print(f"Mean Test loss of symbolic formula: {ts_mean}")
print(f"Var Test loss of symbolic formula: {ts_var}")
print(f"Std Test loss of symbolic formula: {ts_std}")


Mean Test loss of symbolic formula: 3.740968168131076e-06
Var Test loss of symbolic formula: 4.763240197448409e-13
Std Test loss of symbolic formula: 6.901623140572375e-07


## Symb Reg


### Biochemical

#### IC=1

In [11]:
# model_path_mpnn = './saved_models_optuna/model-biochemical-mpnn/biochemical_mpnn_ic1_s5_pd_mult_12/0'
model_path_mpnn = './saved_models_optuna/model-biochemical-llc/biochemical_llc_2/0'
post_process_mpnn(
    config=bio_config,
    model_path=model_path_mpnn,
    test_set=BIO,
    device='cuda',
    sample_size=10000,
    message_passing=False,
    include_time=False,
    atol=1e-5,
    rtol=1e-5,
    method="dopri5",
    model_type="LLC"
)

Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'accuracy', 'param': 100, 'valid_loss': 0.0002031111653195694}
Fitting G_Net...
Execution time: 15.483906 seconds

Fitting H_Net...
Execution time: 20.911927 seconds
\sum_{j}(-0.5*x_i*x_j) - 0.5 x_{i} + 1
Mean Test loss of symbolic formula: 0.00014416437140122676
Var Test loss of symbolic formula: 6.089843222107333e-10
Std Test loss of symbolic formula: 2.467760770842128e-05
Evaluate raw model

Number of model's parameters: 3780

Mean Test loss of best model: 0.00018182010777915517
Var Test loss of best model: 1.4177396538314576e-11
Std Test loss of best model: 3.7652883738585783e-06


#### SNR

In [26]:
model_paths = [
    "./saved_models_optuna/model-biochemical-llc/biochemical_llc_70db/0",
    "./saved_models_optuna/model-biochemical-llc/biochemical_llc_50db/0",
    "./saved_models_optuna/model-biochemical-llc/biochemical_llc_20db/0"
]

for model_path in model_paths:
    print(model_path)
    
    post_process_mpnn(
        config=bio_config,
        model_path=model_path,
        test_set=BIO,
        device='cuda',
        sample_size=10000,
        message_passing=False,
        include_time=False,
        atol=1e-5,
        rtol=1e-5,
        method="dopri5",
        model_type="LLC"
    )

./saved_models_optuna/model-biochemical-llc/biochemical_llc_70db/0
Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'accuracy', 'param': 100, 'valid_loss': 0.001732065575197339}
Fitting G_Net...
Execution time: 16.035280 seconds

Fitting H_Net...
Execution time: 19.481904 seconds
\sum_{j}(-0.5*x_i*x_j) + e^{- 0.6 x_{i}}
Mean Test loss of symbolic formula: 0.003316562001903852
Var Test loss of symbolic formula: 1.0398403883508865e-07
Std Test loss of symbolic formula: 0.00032246556224671286
Evaluate raw model

Number of model's parameters: 7084

Mean Test loss of best model: 0.0012739889013270538
Var Test loss of best model: 4.247036749303384e-0

### Kuramoto

#### IC=1

In [12]:
model_path_mpnn = './saved_models_optuna/model-kuramoto-llc/kuramoto_llc_2/0'

post_process_mpnn(
    config=kur_config,
    model_path=model_path_mpnn,
    test_set=KUR,
    device='cuda',
    sample_size=10000,
    message_passing=False,
    include_time=False,
    atol=1e-5,
    rtol=1e-5,
    method="dopri5",
    model_type="LLC"
)

Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'score', 'param': 200, 'valid_loss': 0.0008288701064884663}
Fitting G_Net...
Execution time: 36.346022 seconds

Fitting H_Net...
Execution time: 42.671804 seconds
\sum_{j}(-0.5*sin(x_i - x_j)) + 2.0
Mean Test loss of symbolic formula: 0.0010122246070144076
Var Test loss of symbolic formula: 8.348958345019636e-10
Std Test loss of symbolic formula: 2.8894564099532e-05
Evaluate raw model

Number of model's parameters: 1852

Mean Test loss of best model: 0.004207481785366933
Var Test loss of best model: 2.372292758651333e-08
Std Test loss of best model: 0.00015402249052172


#### SNR

In [27]:
model_paths = [
    "./saved_models_optuna/model-kuramoto-llc/kuramoto_llc_70db/0",
    "./saved_models_optuna/model-kuramoto-llc/kuramoto_llc_50db/0",
    "./saved_models_optuna/model-kuramoto-llc/kuramoto_llc_20db/0"
]

for model_path in model_paths:
    print(model_path)
    
    post_process_mpnn(
        config=kur_config,
        model_path=model_path,
        test_set=KUR,
        device='cuda',
        sample_size=10000,
        message_passing=False,
        include_time=False,
        atol=1e-5,
        rtol=1e-5,
        method="dopri5",
        model_type="LLC"
    )

./saved_models_optuna/model-kuramoto-llc/kuramoto_llc_70db/0
Black-Box fitting 



Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'score', 'param': 50, 'valid_loss': 0.015590826980769634}
Fitting G_Net...
Execution time: 13.881455 seconds

Fitting H_Net...
Execution time: 14.721405 seconds
\sum_{j}(-0.51*sin(x_i - x_j)) + \frac{1.96 x_{i} + 2.21}{x_{i} + 1.01}
Mean Test loss of symbolic formula: 0.01495103258639574
Var Test loss of symbolic formula: 1.5112871813201488e-06
Std Test loss of symbolic formula: 0.0012293442078279576
Evaluate raw model

Number of model's parameters: 1548

Mean Test loss of best model: 0.028366779287656147
Var Test loss of best model: 2.924684110390155e-05
Std Test loss of best model: 0.005408034865263125
./saved_mod

### Epidemics

#### IC=1

In [14]:
model_path_mpnn = './saved_models_optuna/model-epidemics-llc/epidemics_llc_2/0'

post_process_mpnn(
    config=epid_config,
    model_path=model_path_mpnn,
    test_set=EPID,
    device='cuda',
    sample_size=10000,
    message_passing=False,
    include_time=False,
    atol=1e-5,
    rtol=1e-5,
    method="dopri5",
    model_type="LLC"
)

Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'score', 'param': 200, 'valid_loss': 0.00014084752183407545}
Fitting G_Net...
Execution time: 29.668427 seconds

Fitting H_Net...
Execution time: 23.308801 seconds
\sum_{j}(x_j*(0.5 - 0.5*x_i)) - 0.5 x_{i}
Mean Test loss of symbolic formula: 0.00012937623735827705
Var Test loss of symbolic formula: 1.0037482407200115e-10
Std Test loss of symbolic formula: 1.0018723674800157e-05
Evaluate raw model

Number of model's parameters: 6420

Mean Test loss of best model: 0.0002892174234148115
Var Test loss of best model: 9.847822550342792e-10
Std Test loss of best model: 3.138124049546607e-05


#### SNR

In [28]:
model_paths = [
    "./saved_models_optuna/model-epidemics-llc/epidemics_llc_70db/0",
    "./saved_models_optuna/model-epidemics-llc/epidemics_llc_50db/0",
    "./saved_models_optuna/model-epidemics-llc/epidemics_llc_20db/0"
]

for model_path in model_paths:
    print(model_path)
    post_process_mpnn(
        config=epid_config,
        model_path=model_path,
        test_set=EPID,
        device='cuda',
        sample_size=10000,
        message_passing=False,
        include_time=False,
        atol=1e-5,
        rtol=1e-5,
        method="dopri5",
        model_type="LLC"
    )

./saved_models_optuna/model-epidemics-llc/epidemics_llc_70db/0
Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'accuracy', 'param': 100, 'valid_loss': 0.003259724471718073}
Fitting G_Net...
Execution time: 17.598539 seconds

Fitting H_Net...
Execution time: 14.949568 seconds
\sum_{j}(x_j*(0.48 - 0.48*x_i)) - 0.47 x_{i}
Mean Test loss of symbolic formula: 0.0032187569886446
Var Test loss of symbolic formula: 6.417374537329996e-08
Std Test loss of symbolic formula: 0.00025332537451526634
Evaluate raw model

Number of model's parameters: 5244

Mean Test loss of best model: 0.00239947228692472
Var Test loss of best model: 2.0914823083154643e-07
St

### Population

#### IC=1

In [15]:
model_path_mpnn = './saved_models_optuna/model-population-llc/population_llc_2/0'

post_process_mpnn(
    config=pop_config,
    model_path=model_path_mpnn,
    test_set=POP,
    device='cuda',
    sample_size=10000,
    message_passing=False,
    include_time=False,
    atol=1e-5,
    rtol=1e-5,
    method="dopri5",
    model_type="LLC"
)

Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'score', 'param': 200, 'valid_loss': 0.000729596009477973}
Fitting G_Net...
Execution time: 29.455979 seconds

Fitting H_Net...
Execution time: 32.436466 seconds
\sum_{j}(0.2*x_j**3) - 0.5 x_{i}
Mean Test loss of symbolic formula: 0.0007106232224032283
Var Test loss of symbolic formula: 4.776991871190479e-09
Std Test loss of symbolic formula: 6.911578597679751e-05
Evaluate raw model

Number of model's parameters: 7980

Mean Test loss of best model: 0.0011213955003768206
Var Test loss of best model: 3.652764800198507e-08
Std Test loss of best model: 0.00019112207617641942


#### SNR

In [29]:
model_paths = [
    "./saved_models_optuna/model-population-llc/population_llc_70db/0",
    "./saved_models_optuna/model-population-llc/population_llc_50db/0",
    "./saved_models_optuna/model-population-llc/population_llc_20db/0"
]

for model_path in model_paths:
    print(model_path)
    post_process_mpnn(
        config=pop_config,
        model_path=model_path,
        test_set=POP,
        device='cuda',
        sample_size=10000,
        message_passing=False,
        include_time=False,
        atol=1e-5,
        rtol=1e-5,
        method="dopri5",
        model_type="LLC"
    )

./saved_models_optuna/model-population-llc/population_llc_70db/0
Black-Box fitting 

Fitting black-box model with score and 50 iterations
Fitting black-box model with score and 100 iterations
Fitting black-box model with score and 200 iterations
Fitting black-box model with accuracy and 50 iterations
Fitting black-box model with accuracy and 100 iterations
Fitting black-box model with accuracy and 200 iterations
Refitting best model with {'model_selection': 'accuracy', 'param': 50, 'valid_loss': 0.001721830340102315}
Fitting G_Net...
Execution time: 11.781712 seconds

Fitting H_Net...
Execution time: 13.001745 seconds
\sum_{j}(0.19*x_j**3) - 0.5 x_{i}
Mean Test loss of symbolic formula: 0.0015994422137737274
Var Test loss of symbolic formula: 2.860996170840082e-08
Std Test loss of symbolic formula: 0.00016914479509698435
Evaluate raw model

Number of model's parameters: 796

Mean Test loss of best model: 0.0021241070547451577
Var Test loss of best model: 1.422478682935696e-08
Std Test 

### Real Epid

In [11]:
model_path_mpnn = './saved_models_optuna/model-real-epid-llc/real_epid_llc_3/0/llc'

pysr_model = lambda : get_pysr_model(
    model_selection="score",
    n_iterations=200,
    parallelism="serial",
    random_state = 9999,
    deterministic = True
)

mpnn_symb, symb_g, symb_h, _ = fit_mpnn(
    model_path=model_path_mpnn,
    pysr_model=pysr_model,
    sample_size=10000,
    message_passing=False
)

In [ ]:
mpnn_symb

\sum_{j}( (x_i - x_j)*exp(-x_j)) + 3.3846776*tanh(x_i + 0.99761075)

: 